In [ ]:
import torch
import torchvision

In [ ]:
from torchvision import transforms

In [ ]:
_tasks = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

In [ ]:
from torchvision.datasets import CIFAR10
cifar = CIFAR10('data', train=True, download=True, transform=_tasks)

Files already downloaded and verified


In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
split = int(0.8 * len(cifar))
index_list = list(range(len(cifar)))
train_idx, valid_idx = index_list[:split], index_list[split:]

In [ ]:
tr_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(valid_idx)

In [ ]:
trainloader = DataLoader(cifar, batch_size=256, sampler=tr_sampler)
validloader = DataLoader(cifar, batch_size=256, sampler=val_sampler)

In [ ]:
import torch.nn.functional as F
import torch.nn as nn

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.linear1 = nn.Linear(1024, 512)
        self.linear2 = nn.Linear(512, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 1024) ## reshaping 
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

model = Model()

In [ ]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay= 1e-6, momentum = 0.9, nesterov = True)

In [ ]:
for epoch in range(1, 31):
    train_loss, valid_loss = [], []
    model.train()
    for data, target in trainloader:
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item()) 
        
    model.eval()
    for data, target in validloader:
        output = model(data)
        loss = loss_function(output, target)
        valid_loss.append(loss.item())
    print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))    

Epoch: 1 Training Loss:  1.3627565803041883 Valid Loss:  1.484304541349411
Epoch: 2 Training Loss:  1.2454202167547432 Valid Loss:  1.3160793691873551
Epoch: 3 Training Loss:  1.1418564790373396 Valid Loss:  1.2300742864608765
Epoch: 4 Training Loss:  1.049852869313234 Valid Loss:  1.2266778707504273
Epoch: 5 Training Loss:  0.9644250881140399 Valid Loss:  1.0420596137642861
Epoch: 6 Training Loss:  0.8987426678086542 Valid Loss:  1.055733396112919
Epoch: 7 Training Loss:  0.8323356983767953 Valid Loss:  1.0087993264198303
Epoch: 8 Training Loss:  0.7749018600791883 Valid Loss:  1.0410875603556633
Epoch: 9 Training Loss:  0.7222372684509132 Valid Loss:  0.9144182115793228
Epoch: 10 Training Loss:  0.6686478469781815 Valid Loss:  1.083518135547638
Epoch: 11 Training Loss:  0.6191151087071486 Valid Loss:  0.8976745322346688
Epoch: 12 Training Loss:  0.5677282815905893 Valid Loss:  0.9332229390740394
Epoch: 13 Training Loss:  0.5126893414053947 Valid Loss:  0.9160177350044251
Epoch: 14 Tr

In [ ]:
dataiter = iter(validloader)
data, labels = dataiter.next()
output = model(data)

In [ ]:
import numpy as np

In [ ]:
_, preds_tensor = torch.max(output, 1)
preds = np.squeeze(preds_tensor.numpy())

In [ ]:
print ("Actual:", labels[:10])
print ("Predicted:", preds[:10])

Actual: tensor([5, 9, 7, 4, 4, 1, 3, 1, 2, 8])
Predicted: [7 1 7 0 3 1 6 9 0 9]
